# MAST30034_Applied Data Science_Project1

## Import Libraries

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium
import os

In [2]:
# Create a spark session
spark = (
SparkSession.builder.appName("MAST30034 project 1")
.config("spark.sql.repl.eagerEval.enabled", True)
.config("spark.driver.memory","10G")
.config("spark.executor.memory","10G")
.config("spark.sql.parquet.cacheMetadata", "true")
.config("spark.sql.session.timeZone", "Etc/UTC")
.getOrCreate()
)

24/08/15 16:23:46 WARN Utils: Your hostname, chumuhandeMacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.13.159.68 instead (on interface en0)
24/08/15 16:23:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 16:23:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read data

Read the raw data of yellow taxi and green taxi

In [6]:
taxi_yellow_raw = spark.read.parquet('../data/raw/NYCTLC_data/yellow_taxi/2023/*.parquet')
taxi_green_raw = spark.read.parquet('../data/raw/NYCTLC_data/green_taxi/2023/*.parquet')

In [8]:
taxi_yellow_raw.show(100)
taxi_yellow_raw.count()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2023-10-01 00:16:44|  2023-10-01 00:16:49|              1|          0.0|         1|                 N|         168|         168|           2|        3.0|  1.0|    0.5|       0.

22123840

24/08/16 04:56:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1116029 ms exceeds timeout 120000 ms
24/08/16 04:56:51 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/16 04:56:55 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

Read the raw data of cityBike

In [24]:
cityBike_data_dir = '../data/raw_data/external_data/2023-citibike-tripdata'
cityBike_file = []

# Read all the csv file in the cityBike_data_dir
for root, dirs, files in os.walk(cityBike_data_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df_spark = spark.read.csv(file_path, header=True, inferSchema=True)
            cityBike_file.append(df_spark)

# Merge all the data together
combined_raw_data = cityBike_file[0]
for data in cityBike_file[1:]:
    combined_raw_data = combined_raw_data.union(data)

In [26]:
combined_raw_data.show(10)
combined_raw_data.count()

+----------------+-------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+-----------+------------+-------------+
|         ride_id|rideable_type|          started_at|            ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|    end_lat|     end_lng|member_casual|
+----------------+-------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+-----------+------------+-------------+
|E33ECB69E2EBC53B| classic_bike|2023-11-25 00:26:...|2023-11-25 00:33:...|  E 5 St & Cooper Sq|         5712.12|Forsyth St & Cana...|       5270.07|         40.72769|        -73.990993| 40.7158155|-73.99422366|       member|
|A1579256C79A4043| classic_bike|2023-11-01 21:27:...|2023-11-01 21:47:...|Calyer St & Jewel St|     

23293647

The raw weather data of New York

In [27]:
weather_raw = spark.read.csv('../data/raw_data/external_data/NYC_weather_2023.csv')
weather_raw.show(10)

+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+----------+-----------+--------------------+
|      _c0|                _c1| _c2|      _c3| _c4|     _c5|   _c6|       _c7|       _c8| _c9|     _c10|    _c11|     _c12|   _c13|            _c14|      _c15|      _c16|          _c17|       _c18|   _c19|      _c20|      _c21|       _c22|                _c23|
+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+----------+-----------+--------------------+
|     name|           datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|c

In [28]:
weather_raw.count()

5138

## Pre-Processing